In [1]:
import rpy2
import sys
sys.path.append('/home/thies/repos/BIU/') # Modify this to where you have downloaded the BIU library
import biu
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.rcParams['svg.fonttype'] = 'none'

import seaborn as sns
import scipy

R = biu.R()
from rdmpy import RDM
RDM.meta(source="repos/IsalaFlow1ReClean/code/0_merge_questionnaires.ipynb")

# Merge the multiple Isala Flow1 questionnaires into 1 table

## Load the questionnaire data

### Q0: The initial website questions

In [2]:
Q0 = biu.formats.XLSX('../data/raw/psuedo_Isala_participants_status_20200917.nopassword.xlsx')['Compact list']
Q0.columns = Q0.iloc[0]
Q0 = Q0.iloc[1:]
Q0['Leeftijd'] = Q0.Leeftijd.apply(lambda x: int(x) if str(x).isdigit() else None)
Q0['idx'] = Q0.Deelnemersnummer
Q0 = Q0.set_index('idx')
Q0.columns = [ 'Q0.%s' % s for s in Q0.columns ]

dups_q0 = [ x[0] for x in biu.ops.lst.freq(Q0.index).items() if x[1] > 1]
print("There were multiple entries for %d participants in Q0." % len(dups_q0))

There were multiple entries for 0 participants in Q0.


### Q2: Questionnaire 1

In [4]:
Q1_excel = biu.formats.XLSX('../data/raw/Q1.qualtrics_raw_pseudo_data_20200811.nopassword.xlsx')
Q1 = Q1_excel['Sheet0']
r0 = Q1.iloc[0]
r1 = Q1.iloc[1]
newnames = [ 'Q1.%s.[%s]' % (a,b) if a != b else 'Q1.%s' % (a) for (a,b) in zip(r0, r1)  ]
Q1 = Q1.rename(columns=dict(enumerate(newnames)))
Q1 = Q1.iloc[3:]
Q1['idx'] = Q1['Q1.ExternalReference.[External Data Reference]'].apply(lambda x: 'ISALA%05d' % int(x))
Q1 = Q1.set_index('idx')

dups_q1 = [ x[0] for x in biu.ops.lst.freq(Q1.index).items() if x[1] > 1]
print("There were multiple entries for %d participants in Q1." % len(dups_q1))

### Q2: Questionnaire 2

In [8]:
Q2_excel = biu.formats.XLSX('../data/raw/Q2.qualtrics_raw_psuedo_data_20210206.nopassword.xlsx')

In [9]:
Q2 = Q2_excel['Sheet0']
r0 = Q2.iloc[0]
r1 = Q2.iloc[1]
newnames = [ 'Q2.%s.[%s]' % (a,b) if a != b else 'Q2.%s' % (a) for (a,b) in zip(r0, r1)  ]
Q2 = Q2.rename(columns=dict(enumerate(newnames)))
Q2 = Q2.iloc[2:-1] # Remove the last line. - NO ISALA NUMBER - Check with sarah about this.
Q2['idx'] = Q2['Q2.ExternalReference.[External Data Reference]']

#### Remove duplicate responses in Q2

In [10]:
# Remove duplicates
dups_q2 = [ x[0] for x in biu.ops.lst.freq(Q2['Q2.ExternalReference.[External Data Reference]']).items() if x[1] > 1]

sel = Q2[Q2['Q2.ExternalReference.[External Data Reference]'].isin(dups_q2) & (Q2['Q2.Finished'] =='True')].sort_values('Q2.ExternalReference.[External Data Reference]')

remove_duplicate_completed = sel.loc[sel['Q2.RecordedDate.[Recorded Date]'] != sel.groupby('Q2.ExternalReference.[External Data Reference]')\
                                      ['Q2.RecordedDate.[Recorded Date]'].transform(lambda x: max(x))]\
                                .sort_values('Q2.ExternalReference.[External Data Reference]').index
#sel

remove_duplicate_incomplete = Q2[Q2['Q2.ExternalReference.[External Data Reference]'].isin(dups_q2) & (Q2['Q2.Finished'] != 'True')].index

Q2 = Q2.loc[[i for i in Q2.index if i not in set(remove_duplicate_completed) | set(remove_duplicate_incomplete) ]]

In [11]:
Q2 = Q2.set_index('idx')

In [12]:
dups_q2 = [ x[0] for x in biu.ops.lst.freq(Q2['Q2.ExternalReference.[External Data Reference]']).items() if x[1] > 1]
print("There were multiple entries for %d participants in Q2." % len(dups_q2))
dups_q2

There were multiple entries for 0 participants in Q2.


[]

## A: Flow 1 arrival data

In [13]:
arrival_flow1 = biu.formats.XLSX('../data/raw/Arrival_flow1_data-analysis_20210206.xlsx')['Blad1']
arrival_flow1.columns = arrival_flow1.iloc[0]
arrival_flow1 = arrival_flow1.iloc[1:3343]

# Correct the Isala IDs
arrival_flow1['Isala ID'] = arrival_flow1['Isala ID'].apply(lambda x: 'ISALA%05d' % int(''.join([l for l in x if l.isdigit()])))

In [14]:
import datetime
def calculate_cycle_day(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    if isinstance(row['Start menstrual cycle'], datetime.datetime) and isinstance(row['Sample date'], datetime.datetime):
        ndays = (row['Sample date'] - row['Start menstrual cycle']).days
        if (ndays > 40) | (ndays < 0):
            return None
        else:
            return ndays
    else:
        return None
    #fi
arrival_flow1['day_of_cycle'] = arrival_flow1.apply(calculate_cycle_day, axis=1)


### Remove duplicate arrival data

In [15]:
# Remove the duplicate samples. Always use the most recently arrived sample
arrival_flow1 = arrival_flow1.loc[(arrival_flow1['Sample date'] == arrival_flow1.groupby('Isala ID')['Sample date'].transform(lambda x: max(x) )) | \
                                 pd.isna(arrival_flow1['Sample date']) ]

In [16]:
dups_arrival = [ x[0] for x in biu.ops.lst.freq(arrival_flow1['Isala ID']).items() if x[1] > 1]
print("There were multiple entries for %d participants in arrival data." % len(dups_arrival))
', '.join(dups_arrival)

There were multiple entries for 0 participants in arrival data.


''

In [17]:
arrival_flow1['idx'] = arrival_flow1['Isala ID']
arrival_flow1 = arrival_flow1.set_index('idx')
arrival_flow1 = arrival_flow1.rename(columns={c: 'A.%s' % c for c in arrival_flow1.columns})

# Merge the data frames

In [18]:
Q_flow1 = Q0.join(Q1).join(Q2).join(arrival_flow1).sort_index()

# Attach filter operations
Q_flow1['Q1'] = [ idx in Q1.index for idx in Q_flow1.index ]
Q_flow1['Q2'] = [ idx in Q2.index for idx in Q_flow1.index ]
Q_flow1['A']  = [ idx in arrival_flow1.index for idx in Q_flow1.index ]
#Q_flow1['16S'] = [ idx in S.participant.values for idx in Q_flow1.index ]
#Q_flow1['16S_QC'] = [ idx in HQS.participant.values for idx in Q_flow1.index ]

# Export the merged dataframe

In [23]:
with RDM('../data/processed/merged_questionnaires.xlsx', 'w', nouser=True) as ofd:
    Q_flow1.to_excel(ofd)
#ewith

with RDM('../data/processed/merged_questionnaires.pkl', 'w', nouser=True) as ofd:
    Q_flow1.to_pickle(ofd)
#ewith

In [22]:
? RDM

Init signature:
 RDM(
    path,
    mode='w',
    file=False,
    nouser=False,
    suffix=True,
    modified=True,
    tell=False,
    **meta,
)
Docstring:      <no docstring>
File:           /mnt/b/thies/miniconda/envs/biu/lib/python3.8/site-packages/rdmpy/core.py
Type:           type
Subclasses:     
